# Итоговый проект на тему "Отслеживание русского языка жестов"
Модель представляет собой программу по отслеживанию языка жестов при помощи видеокамеры,
данные собирались во время использования модели самой программой.

Обучение модели происходило на сайте: https://teachablemachine.withgoogle.com/train/image

Используемые версии tensorflow и keras: 2.7.0


При работе модель будет отображать видео с веб-камеры и обводить прямоугольником кисть с распознанным жестом над которым будет указано название найденного жеста. При зажатии кнопки 'F' жест, не сменяемый на изображении в течение 15 кадров будет записан в переменную, что позволяет после завершения программы увидеть готовую строку из введенных жестов.


Дальнейшее развитие может заключаться в применении модели для разбора языка жестов на предзаписанном видео-файле, например при помощи бота telegramm, это позволит людям не знакомым с языком жестов понимать людей его использующих, что улучшит пользовательский опыт людей с ограниченными возможностями при использовании Мессенджера.

In [ ]:
import tensorflow
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
 
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier("Model/keras_model.h5", "Model/labels.txt")
 
offset = 20
imgSize = 300
labels = ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'С', 'Ы', ' ', 'И', "Й", "К", "Л", "Н", "М", "О", "П", "Р", "Т",
          "У", "Ф", "Х", "Ц", "Ч","Ш", "Щ", "Ъ", "Ь", "Э", "Ю", "Я"]
text = ''
warn = 0
frameNum =0
while True:
    success, img = cap.read()
#   Копирование изначального изображения с камеры, чтобы не отправить видимые соединения для предикта модели
#   Линии и точки будут рисоваться на отдельном изображении кисти, на основном окне их не будет.
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
        imgCropShape = imgCrop.shape
 
        aspectRatio = h / w
        try:
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
        except:
            if warn == 15:
                print('За областью отслеживания!')
                warn = 0
            
#       Обвод кисти в прямоугольник и отображение найденной буквы
        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)
        
#       Если жест не изменялся 45 кадров, запоминает его как введенный
    if key == ord('f'):
        if labels[index] and labels[index] == labels[index]:
            if frameNum < 10:
                frameNum += 1
            else:
                frameNum = 0
                text = text + str(labels[index])
                print(labels[index])        

#       Можно раскомментировать стору, чтобы увидеть как модель классифицирует кисть
    cv2.imshow("ImageWhite", imgWhite)
    cv2.imshow("Image", imgOutput)
    key = cv2.waitKey(1)
#   Кнопка Q завершает отслеживание и закрывает окна  
    if key == ord('q'):
        break
print(text)
cap.release()
cv2.destroyAllWindows()

# Данная часть отвечает за создание данных для обучения модели

при нажатии на кнопку 'S' программа будет находить кисть, выделять ее в отдельное транслируемое окно и сохранять каждый кадр в выбранной папке.
Данный метод использовался для обученя модели в связи с отсутствием существующих наборов данных с русским сурдо-алфавиом.
Данные для обучения модели собирались вручную и обрабатывались сервисом https://teachablemachine.withgoogle.com/train/image.

cvzone - переделанная на основе библиотеки media-pipe от google, библиотека для распознавания кистей рук, была использована в связи с наличием удобных классов классификатора с возможностью подключения модели и работой с меньшей загрузкой системы.

In [16]:
# установка и импорт библиотек
# !pip install cvzone
# !pip install tensorflow==2.7.0
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math

# Создание класса захвата видео с камеры
cap = cv2.VideoCapture(0)
# Создание класса распознавания рук (ограничение в 1 руку)
detector = HandDetector(maxHands=1)

# Константы отступа для изображения руки и размера изображения
offset = 20
imgSize = 256

# Константы папки с картинками для модели и обнуление счетчика кадров и ошибок
folder = "data/31"
counter = 0
warn = 0

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

        imgCropShape = imgCrop.shape

#       Соотношение ширины к длине изображения кисти
        aspectRatio = h / w

#       Подстройка изображения под заданные размеры, чтобы оно умещалось в окно на белом фоне
#       Использовалось для обучения модели и распознавания жестов
#       конструкция try используется для предотвращения вылета программы
#       при выходе за края области отслеживания
        try:
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize

            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
        except:
            warn += 1
            if warn == 15:
                print('За областью отслеживания!')
                warn = 0
            pass
#       выводит в отдельное окно рамку с определенным изображением кисти
        cv2.imshow("ImageWhite", imgWhite)

    cv2.imshow("Image", img)

#   При зажатии 'S' программа будет сохранять изображения руки в папку с data
#   Директории в которой расположен блокнот, использовалась для обучения модели
    key = cv2.waitKey(1)
    if key == ord("s"):
#       Счетчик сделанных кадров
        counter += 1
        cv2.imwrite(f'{folder}/{counter}.jpg', imgWhite)
        print(counter)
        
#   Кнопка Q завершает отслеживание и закрывает окна  
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
